In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from collections import defaultdict

In [2]:
def resample(df, filename):
    if 'Timestamp' not in df.columns or 'Whole_System_Energy_Consumed' not in df.columns:
        print(f"Missing columns in {filename}")
        return None
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    df.set_index('Timestamp', inplace=True)
    df['actual consumption'] = df['Whole_System_Energy_Consumed'].diff().fillna(0)
    hourly = df['actual consumption'].resample('H').sum()
    return hourly.to_frame(name='actual consumption')

In [ ]:
directory = 
all_profiles = []

In [ ]:
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        df = pd.read_csv(os.path.join(directory, filename))
        hourly = resample(df, filename)
        if hourly is not None:
            full_index = pd.date_range(start=hourly.index.min(), end=hourly.index.max(), freq='H')
            hourly = hourly.reindex(full_index, fill_value=0)
            avg_profile = hourly.groupby([hourly.index.month, hourly.index.hour]).mean()
            flat = avg_profile.unstack(level=0).T
            flat['House'] = filename
            all_profiles.append(flat)